# TODO: 
1. have this spit out a proposed meal. 
2. In natural text, feed to chatgpt and ask it to rate
3. parse the rating and incorporate as reward.


In [26]:
import openai
import pandas as pd
import re

prompt_types = {
    'Lucas': 'I am a 6ft tall 25 year old male, 168 pounds. I am very active.\
        Please rate the following proposed meal on a scale of 1-10, where\
              1 is an unhealthy or unnatural and 10 is a perfect, balanced healthy meal.\n',
}

def create_prompt(meal_selections, person_type='Lucas'):
    prompt = prompt_types[person_type]
    for meal_type, meal_value in meal_selections.items():
        prompt += f'{meal_type}: {meal_value}\n'
    prompt += 'Please explain your answer, but end with a single numeric rating.'
    return prompt

def parse_meal_rating(response):
    # try to find a number in the response
    try:
        # 'a X out of 10'
        num = re.search(r'\d+',re.search(r'\d+ out of \d+', response).group()).group()
    except:
        try:
            # 'a X/10'
            num = re.search(r'\d+',re.search(r'\d+/\d+', response).group()).group()
        except:
            # 'a X'
            num = re.search(r'\d+', response).group()
        
    return int(num)

with open('gpt_key.txt', 'r') as file:
    openai.api_key = file.read().rstrip()

def get_reward_from_chatgpt(proposed_meal, verbose=False):
    """
    Given a proposed meal, get a reward from the chatgpt model.
    """
    resp = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a helpful assistant and an expert in nutrition and meal planning."},
                {"role": "user", "content": create_prompt(proposed_meal)},
            ]
    )
    text = resp['choices'][0]['message']['content']
    if verbose:
        print(text)
    return parse_meal_rating(text)



In [27]:
sample_meal_set = {'breakfast': 'eggs and bacon', 'lunch': 'chicken and rice', 'dinner': 'steak and potatoes'}
print(create_prompt(sample_meal_set))

I am a 6ft tall 25 year old male, 168 pounds. I am very active.        Please rate the following proposed meal on a scale of 1-10, where              1 is an unhealthy or unnatural and 10 is a perfect, balanced healthy meal.
breakfast: eggs and bacon
lunch: chicken and rice
dinner: steak and potatoes
Please explain your answer, but end with a single numeric rating.


In [28]:
print(get_reward_from_chatgpt(sample_meal_set))


6


In [33]:
from dietkit import load_ingredient, load_menu, load_diet, load_sample_criteria
import pandas as pd

sample_ingredients = load_ingredient(sample_language = 'eng')
sample_menus = load_menu(ingredients = sample_ingredients, sample_language = 'eng')
sample_diets = load_diet(menus = sample_menus, num_loads = 100, sample_language = 'eng', sample_name = 'ML')

def get_ingredient_category(ingredient_name, ingredients):
    return ingredients[ingredient_name].category

all_categories = set([get_ingredient_category(ingredient, sample_ingredients) for ingredient in sample_ingredients])

# convert sample menus to a dataframe
sample_menus_df = pd.DataFrame.from_dict(sample_menus, orient = 'index')

# rename sample menus column 0 to 'menu object'
sample_menus_df.rename(columns = {0: 'object'}, inplace = True)

# add a column for the menu category
sample_menus_df['category'] = sample_menus_df['object'].apply(lambda x: x.category)
# convert sample_menus_df category column to string
sample_menus_df['category'] = sample_menus_df['category'].astype(str)

# get nutrition from object
# sample_menus_df.iloc[0]['menu object'].nutrition

def sample_from_menu_category(sample_menus_df, category):
    print(category)
    return sample_menus_df[sample_menus_df['category'] == category].sample(1)

In [34]:


%     K: arms

%     TSeq: horizon

%     M: number of batches

%     b = T^(1/M); TGridAdaptive = floor(b.^(1:M));...,

%         TGridAdaptive = floor(TGridAdaptive/K) * K; TGridAdaptive(M) = T; ...,

%         TGridAdaptive = [0,TGridAdaptive]; % adaptive batch grids

%     a = T^(1/(2 - 2^(1-M))); TGridMinimax = floor(a.^(2.-1./2.^(0:M-1)));...,

%     TGridMinimax(M) = T; ...,

%     TGridMinimax = [0,TGridMinimax]; % minimax batch grids    

%     mu: batch mean

%     gamma: tunning parameter

UsageError: Line magic function `%` not found.


In [35]:
arms = ['Fruits (snack)',
 'Salad (snack)',
 'Milk (snack)',
 'Drink (snack)',
 'Combo meal (snack)',
 'Soup (snack)',
 'Nuts (snack)',
 'Grains (snack)',
 'Cereal (snack)',
 'Rice',
 'Noodles',
 'Combo meal rice',
 'Soup',
 'Salad',
 'Deep fried',
 'Braised',
 'Grilled',
 'Steamed',
 'Stir-fried',
 'Pancake',
 'Kimchi']

In [36]:
def parse_nutrition_into_string(nutrition):
    nutrition_string = ''
    for key in nutrition:
        nutrition_string += key + ': ' + str(nutrition[key]) + ', '
    return nutrition_string

def sample_meal_set_from_arms(arms, sample_menus_df):
    meal_set = {}
    for i in range(len(arms)):
        # if arms[i] contains 'snack', remove 'S' from name
        meal = sample_from_menu_category(sample_menus_df, arms[i]).iloc[0]['object']
        meal_string = meal.name.replace('S ', '') + ' (' + parse_nutrition_into_string(meal.nutrition) + ')'
        meal_set['meal ' + str(i)] = meal_string
    return meal_set

sample_meal_set_from_arms(arms, sample_menus_df)

Fruits (snack)
Salad (snack)
Milk (snack)
Drink (snack)
Combo meal (snack)
Soup (snack)
Nuts (snack)
Grains (snack)
Cereal (snack)
Rice
Noodles
Combo meal rice
Soup
Salad
Deep fried
Braised
Grilled
Steamed
Stir-fried
Pancake
Kimchi


{'meal 0': 'nectarine -60g (Energy: 19.2, Protein: 0.558, Fat: 0.03, Carbohydrate: 4.71, Total Dietary: 2.28, Calcium: 3.6, Iron: 0.114, Sodium: 0.0, Vitamin A: 4.6999999998, Vitamin B1 (Thiamine): 0.0168, Vitamin B2 (Rivoflavin): 0.010799999999999999, Vitamin C: 1.014, Linoleic Acid: 10.631999999999998, Alpha-Linolenic Acid: 3.378, )',
 'meal 1': 'cucumber stick (Energy: 4.2, Protein: 0.366, Fat: 0.006, Carbohydrate: 0.915, Total Dietary: 0.21, Calcium: 5.4, Iron: 0.06, Sodium: 0.9, Vitamin A: 1.5249999999, Vitamin B1 (Thiamine): 0.0006, Vitamin B2 (Rivoflavin): 0.0102, Vitamin C: 3.375, Linoleic Acid: 2.457, Alpha-Linolenic Acid: 2.0429999999999997, )',
 'meal 2': 'milk (100ml) (Energy: 65.0, Protein: 3.08, Fat: 3.32, Carbohydrate: 5.53, Total Dietary: 0.0, Calcium: 113.0, Iron: 0.05, Sodium: 36.0, Vitamin A: 55.0, Vitamin B1 (Thiamine): 0.021, Vitamin B2 (Rivoflavin): 0.162, Vitamin C: 0.79, Linoleic Acid: 73.78, Alpha-Linolenic Acid: 7.6, )',
 'meal 3': 'barley tea (Energy: 0.0, Pr

In [39]:
import numpy as np
np.random.seed(1)

def BASEFunc_custom(arms, sample_menus_df, T=20, M=10, gridType='geometric', gamma=0.5):
    """
    K = number of arms (number of big diet categories)
    T = horizon
    M = number of batches during explore phase
    gridType = 'minimax', 'geometric', or 'arithmetic'
    gamma = tunning parameter
    """
    # get the number of arms
    K = len(arms)
    if gridType == 'minimax':
        a = T**(1/(2 - 2**(1-M)))
        TGrid = np.floor(a**(2 - 1/2**(np.arange(0, M))))
        TGrid[M-1] = T
        TGrid = np.concatenate(([0], TGrid))  # minimax batch grids
    elif gridType == 'geometric':
        b = T**(1/M)
        TGrid = np.floor(b**(np.arange(0, M)))
        TGrid[M-1] = T
        TGrid = np.concatenate(([0], TGrid))  # geometric batch grids
    elif gridType == 'arithmetic':
        TGrid = np.floor(np.linspace(0, T, M+1))
    
    # initialization
    activeSet = np.ones(K, dtype=bool)
    numberPull = np.zeros(K)
    averageReward = np.zeros(K)

    for i in range(0, M):
        availableK = np.sum(activeSet)
        pullNumber = max(np.floor((TGrid[i] - TGrid[i-1])/availableK), 1)
        TGrid[i] = availableK * pullNumber + TGrid[i-1]

        for j in np.where(activeSet)[0]:
            # generate sample_meal_set
            sample_meal_set = sample_meal_set_from_arms([arms[j]], sample_menus_df)
            reward = get_reward_from_chatgpt(sample_meal_set)
            averageReward[j] = averageReward[j] * (numberPull[j]/(numberPull[j] + pullNumber)) + (reward * pullNumber/(numberPull[j] + pullNumber))
            # NOTE: can't use the true mu here, because we don't know the true mu
            numberPull[j] += pullNumber
        
        maxArm = np.max(averageReward[activeSet])
        for j in np.where(activeSet)[0]:
            if (maxArm - averageReward[j]) >= np.sqrt(gamma * np.log(T*K) / numberPull[j]):
                activeSet[j] = False
        
    
    return averageReward, numberPull, activeSet

In [40]:
BASEFunc_custom(arms, sample_menus_df)

Fruits (snack)
Salad (snack)
Milk (snack)
Drink (snack)
Combo meal (snack)
Soup (snack)
Nuts (snack)
Grains (snack)
Cereal (snack)
Rice
Noodles
Combo meal rice
Soup
Salad
Deep fried
Braised
Grilled
Steamed
Stir-fried
Pancake
Kimchi
Soup (snack)
Cereal (snack)
Noodles
Combo meal rice
Soup
Braised
Stir-fried
Kimchi
Soup (snack)
Combo meal rice
Braised
Stir-fried
Kimchi
Soup (snack)
Combo meal rice
Braised
Stir-fried
Braised
Braised
Braised
Braised
Braised
Braised


(array([3.        , 1.        , 3.        , 1.        , 5.        ,
        6.5       , 2.        , 4.        , 6.        , 4.        ,
        7.        , 6.5       , 5.        , 4.        , 3.        ,
        6.7       , 6.        , 6.        , 6.5       , 5.        ,
        6.33333333]),
 array([ 1.,  1.,  1.,  1.,  1.,  4.,  1.,  1.,  2.,  1.,  2.,  4.,  2.,
         1.,  1., 10.,  1.,  1.,  4.,  1.,  3.]),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False,
        False, False, False]))